In [6]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.neighbors import NearestNeighbors
import folium

In [2]:
# Charger le dataset
df = pd.read_csv('Ordered_NLP_preprocessed_df.csv')  

# Initialiser les stopwords
nltk.download('stopwords')
stop_word = stopwords.words('french')
stop_word += stopwords.words('english')

[nltk_data] Downloading package stopwords to /Users/mac/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
# Définition des fonctions de prétraitement
def tok(sentence):
    return nltk.word_tokenize(sentence.lower())

def no_stop(tokens):
    return [token for token in tokens if (token not in stop_word)]

def stem(tokens, language= 'french'):
    stemmizer = SnowballStemmer(language=language)
    return [stemmizer.stem(token) for token in tokens]

def lemmatize(tokens):
    lemmatizer = nltk.WordNetLemmatizer()
    return [lemmatizer.lemmatize(token) for token in tokens]

def preprocess(sentence, stemm=True, lemm=True, stop=True):
    tokens = tok(sentence)
    if stop:
        tokens = no_stop(tokens)
    if lemm:
        tokens = lemmatize(tokens)
    if stemm and not lemm:  # On ne fait pas le stemming si la lemmatisation est activée
        tokens = stem(tokens)
    return ' '.join(tokens)


In [7]:
# Nettoyaer les colonnes et créer la colonne 'Soop' qui contient les parametres souhaités pour notre entrainement

# Fonction pour nettoyer les valeurs en excluant les chiffres et les virgules
def clean_value(value):
    if pd.isna(value):  # Vérifie si la valeur est NaN
        return ""
    cleaned_value = re.sub(r'[\d,]', '', str(value))  # Supprime les chiffres et les virgules
    return cleaned_value.strip()  # Supprime les espaces en début et fin de chaîne

# Colonnes à utiliser pour créer la colonne 'Soop'
colonnes_utilisees = ['Processed_nom_festival', 'Processed_Type', 'Processed_Region',
                      'Processed_Ville', 'Procced_musique', 'Processed_Spectacle_vivant',
                      'Processed_Cinema_audiovisuel', 'Processed_Livre_litterature']

# Créer des colonnes temporaires nettoyées
for col in colonnes_utilisees:
    df[f'{col}_clean'] = df[col].apply(clean_value)

# Créer la nouvelle colonne 'Soop' en combinant les valeurs des colonnes temporaires nettoyées
df['Soop'] = df[[f'{col}_clean' for col in colonnes_utilisees]].apply(lambda row: ' '.join(row.values), axis=1)

# Supprimer les colonnes temporaires nettoyées 
df.drop(columns=[f'{col}_clean' for col in colonnes_utilisees], inplace=True)


In [8]:
# Preprocessing les données
df['Cleaned_Soop'] = df['Soop'].apply(lambda x: preprocess(x, stemm=True, lemm=False))

# Initialisation des vectorizers et transformation des données
bow_vectorizer = CountVectorizer()
tfidf_vectorizer = TfidfVectorizer()

X = df['Cleaned_Soop']

bow_vectorizer.fit(X)
tfidf_vectorizer.fit(X)

X_bow = bow_vectorizer.transform(X)
X_tfidf = tfidf_vectorizer.transform(X)


In [9]:
# Initialisation et entraînement du modèle KNN
modelNN_t = NearestNeighbors(n_neighbors=3, metric='cosine')
modelNN_t.fit(X_tfidf)


NearestNeighbors(metric='cosine', n_neighbors=3)

In [15]:
# Définir la fonction pour trouver les voisins les plus proches
def find_closest_neighbors(query, vectorizer, knn_model, df, n_neighbors=5):
    # Preprocessing la requête
    query_clean = preprocess(query, stemm=True, lemm=False)
    
    # Transformer la requête en TF-IDF
    query_tfidf = vectorizer.transform([query_clean]).toarray()
    
    # Trouver les entrées les plus proches
    distances, indices = knn_model.kneighbors(query_tfidf, n_neighbors=n_neighbors)
    
    # Extraire les entrées similaires
    similar_neighbors = df.iloc[indices[0]]
    return similar_neighbors

# Créer une carte centrée sur la France
def create_map(similar_neighbors):
    carte = folium.Map(location=[46.603354, 1.888334], zoom_start=6)

    # Ajouter des marqueurs pour chaque position géographique dans les entrées similaires
    for index, row in similar_neighbors.iterrows():
        geocode = row['Geocode'].split(',')
        if len(geocode) == 2:  # Vérifiez que vous avez exactement deux parties après split
            lat = float(geocode[0].strip())
            lon = float(geocode[1].strip())
            nom_festival = row['Processed_nom_festival']
            folium.Marker(location=[lat, lon], popup=nom_festival).add_to(carte)
    
    # Enregistrer et afficher la carte
    carte.save("carte_france.html")
    return carte


In [18]:
# Demander à l'utilisateur d'entrer une requête
query = input("Entrez une description du festival : ")

# Trouver les voisins similaires
closest_neighbors = find_closest_neighbors(query, tfidf_vectorizer, modelNN_t, df, n_neighbors=5)

# Afficher les résultats
print("Les entrées les plus proches :")
closest_neighbors

# Créer et afficher la carte avec les positions des festivals similaires
carte = create_map(closest_neighbors)
carte


Les entrées les plus proches :
